# Building Machine Learning Classifiers: Random Forest on a holdout test set

### Read in & clean text

In [6]:
import nltk
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer
import string

stopwords = nltk.corpus.stopwords.words('english')
ps = nltk.PorterStemmer()

data = pd.read_csv("SMSSpamCollection.tsv", sep='\t')
data.columns = ['label', 'body_text']

def count_punct(text):
    count = sum([1 for char in text if char in string.punctuation])
    return round(count/(len(text) - text.count(" ")), 3)*100

data['body_len'] = data['body_text'].apply(lambda x: len(x) - x.count(" "))
data['punct%'] = data['body_text'].apply(lambda x: count_punct(x))

def clean_text(text):
    text = "".join([word.lower() for word in text if word not in string.punctuation])
    tokens = re.split('\W+', text)
    text = [ps.stem(word) for word in tokens if word not in stopwords]
    return text

tfidf_vect = TfidfVectorizer(analyzer=clean_text)
X_tfidf = tfidf_vect.fit_transform(data['body_text'])

X_features = pd.concat([data['body_len'], data['punct%'], pd.DataFrame(X_tfidf.toarray())], axis=1)
X_features.columns = X_features.columns.astype(str)
X_features.head()

,body_len,punct%,0,1,2,3,4,5,6,7,...,8094,8095,8096,8097,8098,8099,8100,8101,8102,8103
0,128,4.7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,49,4.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,62,3.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,28,7.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,135,4.4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Explore RandomForestClassifier through Holdout Set

In [7]:
from sklearn.metrics import precision_recall_fscore_support as score
from sklearn.model_selection import train_test_split

In [8]:
X_train, X_test, Y_train, Y_test = train_test_split(X_features, data['label'],test_size=0.2)

#X_train contains 80% of your features data (for training the model)
#Y_train contains the corresponding labels for the data in X_train

#X_test contains the other 20% of your features data (for testing the model)
#Y_test contains the corresponding labels for the data in X_test.

In [9]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_estimators=50, max_depth=20, n_jobs=-1)
rf_model = rf.fit(X_train, Y_train)

In [10]:
sorted(zip(rf_model.feature_importances_, X_train.columns), reverse=True) # gives the importance of each feature

[(0.03971797489979465, '3134'),
 (0.038515297885552406, 'body_len'),
 (0.030188840055775667, '2031'),
 (0.028151283948348323, '5724'),
 (0.027891917329669644, '7350'),
 (0.022301526314541305, '3443'),
 (0.020931726318120618, '7218'),
 (0.020508323770344554, '6746'),
 (0.020486508294164325, '4796'),
 (0.018630866883093913, '5988'),
 (0.017423813215206776, '2171'),
 (0.017069279026782947, '1803'),
 (0.016335599645154502, '6285'),
 (0.015698546624494682, '5078'),
 (0.015515530855358017, '2299'),
 (0.014737587345396631, '7027'),
 (0.014686756128517076, '5453'),
 (0.012994710469037291, '7461'),
 (0.0120689235792967, '392'),
 (0.010931484887221154, '1881'),
 (0.010849195779565871, '690'),
 (0.01066932306059286, '1228'),
 (0.010541839415843536, '7782'),
 (0.008853167129673212, '397'),
 (0.008313999131214812, '1941'),
 (0.007995567869230624, '6971'),
 (0.007962947775018013, '6050'),
 (0.00737823276292485, '294'),
 (0.00718119074105447, '4269'),
 (0.007111929143493184, '354'),
 (0.0068434435199

In [20]:
Y_pred = rf_model.predict(X_test)
precision, recall, fscore, support = score(Y_test, Y_pred, pos_label='spam', average='binary') # gives the precision, recall, fscore and support


In [21]:
print('Precision: {} / Recall: {} / Accuracy: {}'.format(round(precision, 3),
                                                        round(recall,3),
                                                        round((Y_pred==Y_test).sum() / len(Y_test),3)))

# precision 1.0 means that all the spam messages were correctly identified as spam
# recall 0.53 means that 53% of the spam messages were correctly identified as spam
# accuracy 0.97 means that 97% of the messages were correctly identified as spam or ham

Precision: 1.0 / Recall: 0.53 / Accuracy: 0.936
